In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

from proc_v2 import p
from ml_v2 import X_cat, X_num, X_all, target, kf, skf, ss, config
from ml_v2 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

2025-05-21 00:53:15.357084: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 00:53:15.366119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747756395.377815   20760 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747756395.381510   20760 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747756395.390805   20760 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Duration-Aware CV-5

In [4]:
from itertools import product
X_o2 = [i + '_mul_'+ j for i, j in product(X_num, X_num) if i < j]
X_o2

['Age_mul_Height',
 'Age_mul_Weight',
 'Age_mul_Duration',
 'Age_mul_Heart_Rate',
 'Age_mul_Body_Temp',
 'Height_mul_Weight',
 'Duration_mul_Height',
 'Duration_mul_Weight',
 'Duration_mul_Heart_Rate',
 'Heart_Rate_mul_Height',
 'Heart_Rate_mul_Weight',
 'Body_Temp_mul_Height',
 'Body_Temp_mul_Weight',
 'Body_Temp_mul_Duration',
 'Body_Temp_mul_Heart_Rate']

In [5]:
[i for i in df_train.columns if i.startswith('Weight')]

['Weight',
 'Weight_r',
 'Weight_log',
 'Weight_sqrt_d',
 'Weight_2',
 'Weight_div_Age',
 'Weight_div_Height',
 'Weight_div_Duration',
 'Weight_div_Heart_Rate',
 'Weight_div_Body_Temp',
 'Weight_2_mul_Age',
 'Weight_2_mul_Height',
 'Weight_3',
 'Weight_2_mul_Duration',
 'Weight_2_mul_Heart_Rate',
 'Weight_2_mul_Body_Temp',
 'Weight_2_div_Age',
 'Weight_2_div_Height',
 'Weight_2_div_Duration',
 'Weight_2_div_Heart_Rate',
 'Weight_2_div_Body_Temp',
 'Weight_div_Age_2',
 'Weight_div_Age_mul_Height',
 'Weight_div_Age_mul_Duration',
 'Weight_div_Age_mul_Heart_Rate',
 'Weight_div_Age_mul_Body_Temp',
 'Weight_div_Height_2',
 'Weight_div_Duration_mul_Height',
 'Weight_div_Duration_2',
 'Weight_div_Duration_mul_Heart_Rate',
 'Weight_div_Heart_Rate_mul_Height',
 'Weight_div_Heart_Rate_2',
 'Weight_div_Body_Temp_mul_Height',
 'Weight_div_Body_Temp_mul_Duration',
 'Weight_div_Body_Temp_mul_Heart_Rate',
 'Weight_div_Body_Temp_2',
 'Weight_div_Age_sqrt',
 'Weight_div_Height_sqrt',
 'Weight_sqrt',
 '

## CB_da_skf5

In [2]:
hparams = {
    'model_params': {'early_stopping_rounds': 50}, 
    'X_num': ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'Heart_Rate_div_Duration', 'Weight_div_Height_2'] + X_o2, 
    'X_cat': ['Sex']
    'validation_fraction': 0.2
}

result = sc.cv_result('cb5_skf5', df_train, skf, hparams, config, cb_adapter, use_gpu = 0, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

id
0         8
1         2
2         1
3         8
4         8
         ..
749995    9
749996    5
749997    9
749998    5
749999    6
Name: duration_bin, Length: 750000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [3]:
df_train['Heart_Rate'].nunique()

63

In [4]:
df_train['Body_Temp'].nunique()

75